<a href="https://colab.research.google.com/github/anik199/House_Prices_Advanced_Regression/blob/main/Autogluon_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
## define configuration
PATH_TRAIN = '/content/train.csv'
PATH_TEST = '/content/test.csv'

PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon.csv'
PATH_EVALML_SUBMISSION = 'submission_evalml.csv'
PATH_FLAML_SUBMISSION = 'submission_flaml.csv'
PATH_H2OAML_SUBMISSION = 'submission_h2oaml.csv'
PATH_LAML_SUBMISSION = 'submission_laml.csv'

MAX_MODEL_RUNTIME_MINS = 15
MAX_MODEL_RUNTIME_SECS = MAX_MODEL_RUNTIME_MINS * 60

In [ ]:
pip install datatable

In [ ]:
## prepare data
import gc
import os
import shutil
import datatable as dt
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

train = dt.fread(PATH_TRAIN)[:100000, :]
test = dt.fread(PATH_TEST)

target = train['SalePrice'].to_numpy().ravel()
test_ids = test['Id']

del train[:, ['Id', 'SalePrice']]
test = test[:, train.names]

In [ ]:
! pip install delayed

In [ ]:
## install packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q -U graphviz
!python3 -m pip install -q scikit-learn==0.24.2

In [ ]:
## import packages
from autogluon.tabular import TabularPredictor

In [ ]:
## run model
train['SalePrice'] = dt.Frame(target)

model_autogluon = TabularPredictor(label='SalePrice')
model_autogluon.fit(train_data=train.to_pandas(), excluded_model_types=['KNN'], time_limit=MAX_MODEL_RUNTIME_SECS)

del train['SalePrice']

In [ ]:
## generate predictions
preds_autogluon = model_autogluon.predict_proba(test.to_pandas())

In [ ]:
## create submission
submission = dt.Frame(Id=test_ids, SalePrice=dt.Frame(preds_autogluon))
submission.to_csv('submission.csv')
submission.head()

In [ ]:
## clear memory
shutil.rmtree('AutogluonModels')
del model_autogluon

gc.collect()